### Get the statistics of a single post  
+ 發文者 - poster_id
+ 評論者計數 - commenter_ids 
+ 反應者計數 - reactor_ids

In [12]:
import facebook_scraper as fs
from collections import Counter

In [13]:
MAX_COMMENTS = True
COOKIES = 'www.facebook.com_cookies.txt'

In [14]:
def calc_commenter(cmt, commenter_ids):
    commenter_ids[cmt['commenter_id']] += 1


def calc_reactor(rcts, reactor_ids):
    for rct in rcts:
        reactor_ids[rct['reactor_id']] += 1


def post_stats(post_id):

    # get the post (this gives a generator)
    gen = fs.get_posts(
        post_urls=[post_id],
        cookies=COOKIES,
        options={"comments": MAX_COMMENTS, "reactors": True}
    )

    # take 1st element of the generator which is the post we requested
    post = next(gen)

    # assign stats units
    poster_id = post['user_id']
    commenter_ids = Counter()
    reactor_ids = Counter()

    # reaction (for post)
    calc_reactor(post['reactors'], reactor_ids) 

    # comment (for post)
    # reaction (for comment)
    cmts = post['comments_full']
    for cmt in cmts:
        calc_commenter(cmt, commenter_ids)
        calc_reactor(cmt['comment_reactors'], reactor_ids)
        for re in cmt['replies']:
            calc_commenter(re, commenter_ids)
            calc_reactor(re['comment_reactors'], reactor_ids)
    
    return poster_id, commenter_ids, reactor_ids

#### Example test  

In [15]:
# https://m.facebook.com/groups/python/permalink/1453403678833525

In [16]:
POST_ID = '1453403678833525'
poster_id, commenter_ids, reactor_ids = post_stats(POST_ID)

print('發文者: ', poster_id)
print('評論計數: ', commenter_ids)
print('反應計數: ', reactor_ids)

發文者:  100026224234527
評論計數:  Counter({'100085894517963': 2, '100089704909573': 2, '100026224234527': 2, '100001177415316': 2, '100003502675716': 1, '100013414364133': 1, '100093154903445': 1, '1163530803': 1, '1096440191': 1, '100003682189096': 1, '100010042593294': 1, '100000056946495': 1, '1139118646': 1, '100001396808812': 1, '100078713574211': 1, '100001971176872': 1, '100037337003371': 1, '100030382610302': 1, '100059131621118': 1, '519314542': 1, '1399157537': 1, '100089004758749': 1, '1215966306': 1, '100062176515490': 1, '643490464': 1})
反應計數:  Counter({'270834543757117': 13, '100094185782627': 2, '100093837468225': 1, '100093549174457': 1, '100093540353927': 1, '100092288889732': 1, '100092032867637': 1, '100091647434608': 1, '100090992821931': 1, '100087896445608': 1, '100082762711475': 1, '100079954586579': 1, '100074871356155': 1, '100068813170538': 1, '100024097732165': 1, '100012582348609': 1, '100009744422726': 1, '100008780791642': 1, '100008300381592': 1, '100007880658

#### **注意! 請先修改 facebook_scraper > extractors.py**
修改 extract_reactors 回傳 原本沒有的 reactor_id  
+ 請於 694 行左右，改動程式碼如下:  

In [17]:
# elem_ = elem.find(f"div>div>div>div._4mq.ext a", first=True)
# if elem_ :
#     reactor_id = str(elem_.attrs.get("id")).split('_')[-1]
# else:
#     url = elem.find("a", first=True).attrs.get("href")
#     reactor_id = re.search(r"id=(\d+)", url).group(1) if re.search(r"id=(\d+)", url) else None

# yield {
#     "reactor_id": reactor_id,
#     "name": elem.find("strong", first=True).text,
#     "link": utils.urljoin(FB_BASE_URL, elem.find("a", first=True).attrs.get("href")),
#     "type": reaction_type,
# }

#### **注意! 請使用新創的空帳戶，避免產生例外狀況**  
+ 如果 reactor 為好友/擁有共同好友，會出現 傳訊息/加好友 等按鈕，導致爬取格式不一